In [66]:
import transformers
print(transformers.__version__)
from datasets import load_dataset
import evaluate
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

4.35.2


In [64]:
df.head()

,Interacted_with,utterance_by_other,utterance_by_chandler
0,Chandler Bing,"Oh, I think this is the episode of Three's Com...","Oh, I think this is the episode of Three's Com..."
1,Chandler Bing,"""So how does it feel knowing you're about to d...","""So how does it feel knowing you're about to d..."
2,Chandler Bing,"Ooh, Lambchop. How old is that sock? If I had ...","Ooh, Lambchop. How old is that sock? If I had ..."
3,Chandler Bing,Poulet passes it up to Leetch!,Poulet passes it up to Leetch!
4,Chandler Bing,'Scuse me.,'Scuse me.


In [68]:
df =  pd.read_csv("Bing_Bing.csv")

df.drop(columns= ['Interacted_with'], inplace = True)

df_train, df_temp = train_test_split( df, test_size = 0.30, random_state = 1)
df_val, df_test = train_test_split(df_temp, test_size = 0.45, random_state = 1)
print(df_train.shape, df_test.shape, df_val.shape)

dataset = {}
dataset['train'] = df_train.to_dict('records')
dataset['val'] = df_val.to_dict('records')
dataset['test'] = df_test.to_dict('records')

(5997, 2) (1157, 2) (1414, 2)


<bound method Dataset.__len__ of Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})>

In [82]:
class ExtendedDict(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def __len__(self):
        return super().__len__()

    @property
    def length(self):
        return self.__len__()
    
dict2 = ExtendedDict(dataset)
dict2.__len__

<bound method ExtendedDict.__len__ of {'train': [{'utterance_by_other': 'Hey.', 'utterance_by_chandler': 'Hey.'}, {'utterance_by_other': "Yeah. You know why? 'Cause you came to me first.", 'utterance_by_chandler': 'Well, I thought that would be the best thing to do.'}, {'utterance_by_other': 'Honey, you can relax. Last night at work, Geoffrey told this really sexist joke. After that, not so funny anymore.', 'utterance_by_chandler': "Really? See that's the thing: you gotta keep it smart, people!"}, {'utterance_by_other': 'I just purchased the La-Z-Boy E-cliner 3000.', 'utterance_by_chandler': "That's awesome! That's great! What made you do it?!"}, {'utterance_by_other': 'So have you called her yet?', 'utterance_by_chandler': "Let her know I like her? What are you, insane? It's the next day! How needy do I want to seem? I'm right, right?"}, {'utterance_by_other': 'Mhmm.', 'utterance_by_chandler': 'You know, I flew a long way t see my loving wife? Is she here by the way?'}, {'utterance_by

In [69]:
dataset['train'][0]

{'utterance_by_other': 'Hey.', 'utterance_by_chandler': 'Hey.'}

In [38]:
import torch
from torch.utils.data import Dataset

class CustomXSumDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        article_text = self.dataframe.iloc[idx]['text']
        summary_text = self.dataframe.iloc[idx]['summary']
        return {"text": article_text, "summary": summary_text}

# Assuming df is your pandas DataFrame with the dataset
dataset = CustomXSumDataset(df)

TypeError: Cannot index by location index with a non-integer key

In [27]:
model_checkpoint = "t5-small"
raw_dataset = load_dataset("xsum")
metric = evaluate.load("rouge")

In [45]:
raw_dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [6]:
raw_dataset['

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [28]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [88]:
max_input_length = 1024
max_target_length = 120
prefix = "summarize"

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["utterance_by_other"]]
    model_inputs = tokenizer(inputs, max_length = max_input_length, truncation = True)
    
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['utterance_by_chandler'], max_length = max_input_length, truncation = True)
        
    model_inputs['labels'] = labels['input_ids']
    return model_inputs



In [92]:
token = {}
token['train'] = [preprocess_function(x) for x in dataset['train'] ] 
token['train'][0]

{'input_ids': [[21603, 566, 1], [21603, 15, 1], [21603, 63, 1], [21603, 5, 1]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]], 'labels': [9459, 5, 1]}

In [31]:
tokenized_datasets = raw_dataset.map(preprocess_function, batched = True)

In [32]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [25]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)